In [1]:
from langchain.tools import tool

# 기본적인 도구 정의

@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.
    
    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

In [ ]:
# 도구 이름 사용자 정의

@tool("web_search") # 커스텀 이름
def search(query: str) -> str:
    """Search the web for information"""
    return f"Results for: {query}"

print(search.name) # web_search

In [ ]:
# 도구 사용자 정의 설명 추가

@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))

In [ ]:
# 고급 스키마 정의

from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel): # 필요한 필드 값 정의
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit perference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )
    
@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [ ]:
# 도구 상태 접근
from langchain.tools import tool, ToolRuntime

@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """Summarize the conversation so far."""
    
    messages = runtime.state["messages"] # messages에 모든 대화 내용을 저장 중
    
    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage") # human의 메세지 수
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")       # ai의 메세지 수
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")   # tool의 메세지 수
    
    return (
        f"Conversation has {human_msgs} user messages, "
        f"{ai_msgs} AI responses, and "
        f"{tool_msgs} tool results."
    )

@tool
def get_user_preference( # 저장된 사용자 설정값을 반환
    pref_name: str,
    runtime: ToolRuntime
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")


In [ ]:
from langgraph.types import Command
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, ToolRuntime


# 모든 대화 기록을 제거하여 대화 업데이트
@tool
def clear_conversation() -> Command:
    """Clear the conversation history."""
    
    return Command(
        update={
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)]
        }
    )
    
# agent state에서 user_name 업데이트
@tool
def update_user_name(
    new_name: str,
    runtime: ToolRuntime
) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

In [6]:
# Context

from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str
    
@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id
    
    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

agent = create_agent(
    "google_genai:gemini-2.5-flash-lite", 
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You ar a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

print(result)

{'messages': [HumanMessage(content="What's my current balance?", additional_kwargs={}, response_metadata={}, id='0ecca160-2b55-4398-abe5-73dc0b24132e'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_account_info', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--7eec218f-a09b-4d88-a6c3-9137ae82ea79-0', tool_calls=[{'name': 'get_account_info', 'args': {}, 'id': '6d682c40-ee5f-43ae-93ef-74d61d0576cc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 46, 'output_tokens': 12, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Account holder: Alice Johnson\nType: Premium\nBalance: $5000', name='get_account_info', id='f14b1c9c-6846-464e-8477-ed659d110e14', tool_call_id='6d682c40-ee5f-43ae-93ef-74d61d0576cc'), AIMessage(content='Your current bal

In [ ]:
# 메모리(저장)

from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

# 메모리에서 해당 user의 user_info 접근
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """Look up user info."""
    store = runtime.store # 프로세스 진행 중 in-memory 공간
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# 메모리에 user_info 저장
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info"

store = InMemoryStore()
agent = create_agent(
    "google_genai:gemini-2.5-flash-lite",
    tools=[get_user_info, save_user_info],
    store=store
)

# 1. user 데이터 저장
agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})

# 2. 유저 정보 불러오기
result = agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})

print(result)

{'messages': [HumanMessage(content="Get user info for user with id 'abc123'", additional_kwargs={}, response_metadata={}, id='fdf49e04-331c-44d2-bd8b-dfb85665d468'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_user_info', 'arguments': '{"user_id": "abc123"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--6124db24-c604-443e-a611-1461104c12ee-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'abc123'}, 'id': 'bdde2b3a-6f48-40f3-b5c7-9daeb12ba23a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 114, 'output_tokens': 22, 'total_tokens': 136, 'input_token_details': {'cache_read': 0}}), ToolMessage(content="{'name': 'Foo', 'email': 'foo@langchain.dev', 'age': 25.0}", name='get_user_info', id='22f53a46-65bc-4c52-8479-441b948f9798', tool_call_id='bdde2b3a-6f48-40f3-b5c7-9daeb12ba

In [ ]:
# steam_writer(tool의 오래걸릴 때 streaming 처리)
from langchain import tool, ToolRuntime

@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """Get weather for a given city."""
    writer = runtime.stream_writer
    
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    
    return f"It's always sunny in {city}!"